# Autopilot Suggest Endpoint
**Important Notices:**
* Please make a **copy** of this file, otherwise other users might be able to access your API key.
* Run the individual codes snippets one-by-one and make sure to follow the instructions from top to bottom.
* There are several placeholders in the code. Make sure to replace these with the required values.

**Functional Overview:**
1. Loads structured inputs read from Excel data file.
2. Builds and sends payloads to the Climatiq Autopilot Suggest API.
3. Handles retries, optional fields, and API errors gracefully.
4. Extracts top N emission factor suggestions per row.
5. Saves results with detailed metadata back to Excel file.

**Installation Requirements:** python3, numpy, pandas

In [65]:
import pandas as pd
import numpy as np
import requests
import time
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

## Configuration
`file_path` (AP_script_sample_data.xlsx) needs to be replaced with the name of your Excel file of input procurement data.

`output_path` (AP_script_output.xlsx) needs to be replaced with the name you would like the output file to be called, which will contain all the matched emission factor data.

`max_suggestions` is where the number of desired emission factor matches per item is defined. Currently set to 5, but this can be any value up to 20.

`REPLACE_WITH_YOUR_KEY` is where your API key needs to be set. You can retreive a personal API key using [these instructions](https://www.climatiq.io/docs/guides/how-tos/getting-api-key).


In [66]:
# Configuration
file_path = "AP_script_sample_data.xlsx" # Input data file name
output_path = "AP_script_output.xlsx" # Desired name of the final output file to be called
max_suggestions = 5 # Adjust this to as many matches per item as required
api_key = "REPLACE_WITH_YOUR_KEY"  # Replace with your API key
authorization_headers = {"Authorization": f"Bearer {api_key}"} # Set Authorisation headers with API key
url = "https://preview.api.climatiq.io/autopilot/v1-preview4/suggest" # URL of the Autopilot Suggest endpoint that the POST request is sent to.

## HTTP Session Setup
Configure retry logic for resilient API calls.

In [67]:
# Setup session with retry logic
session = requests.Session()
retries = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://", adapter)

## Load and Prepare Data
Read and rename input columns to match expected API schema.
The columns in your file need to have the same names as the **capitalized names** below.

For example, your procured items column name needs to be "`TEXT`", unit type needs to be re-named to "`UNIT_TYPE`" and so forth.

Alternatively, you can change the capitalized column names in the code below to match your existing column header names.

In [68]:
# Load and clean data
input_df = pd.read_excel(file_path)
input_df = input_df.rename(columns={
    'TEXT': 'text',
    'MODEL': 'model', # Optional, delete if not used
    'UNIT_TYPE': 'unit_type', # Can be Weight, Spend, Volume or Number, one or multiple possible
    'YEAR': 'year', # Optional, delete if not used
    'REGION': 'region', # Optional, delete if not used
    'REGION_FALLBACK': 'region_fallback', # Optional, delete if not used
    'SOURCE': 'source', # Optional, delete if not used
    'EXCLUDE_SOURCE': 'exclude_source', # Optional, delete if not used
    'LCA_ACTIVITY': 'source_lca_activity' # Optional, delete if not used
})
input_df = input_df.replace(np.nan, '')

## Construct Output Columns
Define and preallocate output columns. The `base_cols` array contains the list of column headers that will be populated with the output results. These can be adapted, based on the results you are interested in outputting.

If you choose to adapt the outputs, it is then essential to update the code in the `enumerate(results[:max_suggestions])` method in the code in the **"Iterate and Construct Payloads"** section of this notebook to align with the new outputs.

These `output_cols` will iterate and print as many times as the number of `max_suggestions` specified, to ensure all outputs have column headers.

In [69]:
# Define output columns
base_cols = ['suggestion_name', 'sector', 'category', 'unit_type', 'source', 'year_relevant', 'year_released',
             'region_name', 'source_lca_activity', 'data_quality_flag']
output_cols = [f"{col}_{i+1}" for i in range(max_suggestions) for col in base_cols]
for col in output_cols:
    input_df[col] = ""

## Iterate and Construct Payloads
Below will define the optional input data parameters (`optional_fields` array). The `text` is defined as a required field.
The for statement will loop through each row of the input and build payloads for API calls. Handle missing values explicitly.

`suggest_payload` will construct the most basic core payload with all the required fields for a given row of data.

### Optional Field Handing

Inject optional fields if valid and available (not empty).
Based on the input type (`year`, `region_fallback` etc), there is type-cast and validation per input field.

Parse the cleaned optional inputs to the API payload.

In [ ]:
# Optional fields to check for in the input data
optional_fields = [
    "model", "unit_type", "year", "region", "region_fallback",
    "source", "exclude_source", "source_lca_activity"
]

# Loop over each row, grab required fields and check for missing data.
for idx, row in input_df.iterrows():
    text = str(row.get("text", "")).strip()

    if not text:
        for col in output_cols:
            input_df.at[idx, col] = "MISSING_REQUIRED_FIELDS"
        continue

    # Construct base payload with 'required' input fields for the Autopilot suggest endpoint.
    suggest_payload = {
        "suggest": {
            "text": text
        },
        "max_suggestions": max_suggestions
    }

    # Check each optional field in the current row and skip if empty, otherwise clean the value (remove white spaces).
    for field in optional_fields:
        value = row.get(field, "")
        if str(value).strip() == "":
            continue

        clean_value = str(value).strip()

        # Type-casting of optional inputs to their correct type
        if field == "year":
            try:
                suggest_payload["suggest"][field] = int(clean_value)
            except ValueError:
                continue

        elif field == "region_fallback":
            suggest_payload["suggest"][field] = clean_value.lower() == "true"

        elif field in ["unit_type", "source", "exclude_source", "source_lca_activity"]:

            # Ensure mutually exclusive for source/exclude_source
            if field == "exclude_source" and "source" in suggest_payload["suggest"]:
                continue
            if field == "source" and "exclude_source" in suggest_payload["suggest"]:
                continue

            values_array = [v.strip() for v in clean_value.split(",") if v.strip()]
            if values_array:
                suggest_payload["suggest"][field] = values_array

        else:
            suggest_payload["suggest"][field] = clean_value

    # Print constructed payload for each row for debugging
    print(f"\nPayload for row {idx}:\n" + json.dumps(suggest_payload, indent=2))

    # Make API request
    try:
        response = session.post(url, headers=authorization_headers, json=suggest_payload)
        response.raise_for_status()
        results = response.json().get("results", [])

    # Error handling for HTTP request
    except requests.exceptions.HTTPError as e:
        print(f"\nAPI Error at row {idx}: {e}")
        print(f"Status Code: {response.status_code}")
        print("Response Body:", response.text)

        # Clear error output messages
        try:
            error_data = response.json()
            error_code = error_data.get("error_code", "")
            if error_code == "no_emission_factors_found":
                for col in output_cols:
                    input_df.at[idx, col] = "NO_MATCH_FOUND"
            else:
                for col in output_cols:
                    input_df.at[idx, col] = "API_ERROR"
        except Exception:
            for col in output_cols:
                input_df.at[idx, col] = "API_ERROR"
                continue
    except Exception as e:
        print(f"\nUnexpected error at row {idx}: {e}")
        for col in output_cols:
            input_df.at[idx, col] = "ERROR"
        continue

    # Printing the output column headers max_suggestion number of times
    # Extract emission factor details from each API suggestion and organize into a data dictionary
    for i, suggestion in enumerate(results[:max_suggestions]):
        ef = suggestion.get("emission_factor", {})
        flags = ef.get("data_quality_flags", [])
        data = {
            "suggestion_name": ef.get("name", ""),
            "sector": ef.get("sector", ""),
            "category": ef.get("category", ""),
            "unit_type": ef.get("unit_type", ""),
            "source": ef.get("source", ""),
            "year_relevant": ef.get("year", ""),
            "year_released": ef.get("year_released", ""),
            "region_name": ef.get("region_name", ""),
            "source_lca_activity": ef.get("source_lca_activity", ""),
            "data_quality_flag": "TRUE" if flags else "FALSE",
            "suggestion_details": suggestion.get("suggestion_details", "").get("label", "")
        }

        # Write each suggestion's data fields into numbered columns for the current row
        for key, val in data.items():
            input_df.at[idx, f"{key}_{i+1}"] = val

    time.sleep(0.2) # Rate limiting delay between API requests

## Export Output
Save final suggestions to Excel file with provided name.

In [71]:
# Save to output Excel file
input_df.to_excel(output_path, index=False)
print(f"\nOutput saved to: {output_path}")


Output saved to: AP_script_output.xlsx
